In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 

In [ ]:
# pitcher

def get_pitcher(url) :
    response = requests.get(url) 
    html = response.text 
    soup = BeautifulSoup(html, "html.parser")
    soup.get('span')
    score=[i.get_text() for i in soup.select('span')]

    team_score = [[],[],[],[],[],[],[],[],[],[]] ; start = [] ; end = []
    start.append(77) ; end.append(100) ; team_score[0] = score[start[0]:end[0]]
    for i in range(1,10) :
        start.append(end[i-1]+8)
        end.append(start[i]+23)
        team_score[i] = score[start[i]:end[i]]
    
    #팀 이름
    team=[i.get_text() for i in soup.select('td')] ; team=team[32::31][:10]

    for i in range(len(team)) :
        if team[i]=='KIA' : team[i]='HT'
        if team[i]=='롯데' : team[i]='LT'
        if team[i]=='두산' : team[i]='OB'
        if team[i]=='키움' : team[i]='WO'
        if team[i]=='삼성' : team[i]='SS'
        if team[i]=='한화' : team[i]='HH'

    #지표들
    var=[i.get_text() for i in soup.select('a')]
    var=var[83:83+22]
    var.insert(0,'ERA')

    for i in range(len(var)) :
        if var[i]=='출장' : var[i]='delete'
        if var[i]=='선발' : var[i]='START_CK'
        if var[i]=='완투' : var[i]='CG_CK'
        if var[i]=='완봉' : var[i]='delete2'
        if var[i]=='승' : var[i]='WLS_WIN'
        if var[i]=='패' : var[i]='WLS_LOSE'
        if var[i]=='홀드' : var[i]='WLS_HOLD'
        if var[i]=='세' : var[i]='WLS_SAVE'
        if var[i]=='이닝' : var[i]='INN2'
        if var[i]=='실점' : var[i]='R'
        if var[i]=='자책' : var[i]='ER'
        if var[i]=='타자' : var[i]='PA'
        if var[i]=='타수' : var[i]='AB'
        if var[i]=='안타' : var[i]='HIT'
        if var[i]=='2타' : var[i]='H2'
        if var[i]=='3타' : var[i]='H3'
        if var[i]=='홈런' : var[i]='HR'
        if var[i]=='볼넷' : var[i]='BB'
        if var[i]=='고4' : var[i]='IB'
        if var[i]=='사구' : var[i]='HP'
        if var[i]=='삼진' : var[i]='KK'
        if var[i]=='투구' : var[i]='BF'
        
    pitcher_2020 = pd.concat([pd.Series(var), pd.Series(team_score[0])], axis=1)

    for i in range(1,10) :
        pitcher_2020 = pd.concat([pitcher_2020, pd.Series(team_score[i])], axis=1)
    
    pitcher_2020.index = pitcher_2020.iloc[:,0]
    pitcher_2020 = pitcher_2020.iloc[:,1:]
    pitcher_2020.columns = team
    pitcher_2020 = pitcher_2020.T
    pitcher_2020 = pitcher_2020.drop(['delete','delete2'], axis=1)
    pitcher_2020['SH'] = 0
    pitcher_2020['GD'] = 0
    pitcher_2020['ERR'] = 0
    pitcher_2020['SB'] = 0
    pitcher_2020['CS'] = 0
    pitcher_2020['WP'] = 0
    pitcher_2020['BK'] = 0
    
    return pitcher_2020

In [ ]:
home_test_x = get_pitcher('http://www.statiz.co.kr/stat.php?mid=stat&re=1&ys=2020&ye=2020&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=5&tr=&cv=&ml=1&sn=30&pa=0&cn=&si=999&si_it=&si_wd=&si_tm=&si_ha=1&si_te=&si_st=&si_as=&si_or=&si_ty=&si_pl=&si_in=&si_on=&si_um=&si_oc=&si_bs=&si_sc=&si_cnt=&si_aft=&si_li=')
away_test_x = get_pitcher('http://www.statiz.co.kr/stat.php?mid=stat&re=1&ys=2020&ye=2020&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=15&o1=ERAP&o2=OutCount&de=1&lr=5&tr=&cv=&ml=1&sn=30&pa=0&cn=&si=999&si_it=&si_wd=&si_tm=&si_ha=2&si_te=&si_st=&si_as=&si_or=&si_ty=&si_pl=&si_in=&si_on=&si_um=&si_oc=&si_bs=&si_sc=&si_cnt=&si_aft=&si_li=')

In [ ]:
# test 데이터에 존재하지 않는 SF 변수 크롤링
# 해당 변수는 home & away로 나뉘지 않아 각 경기 횟수의 비율로 SF 계산

url = "https://www.koreabaseball.com/Record/Team/Pitcher/Basic2.aspx"
response = requests.get(url) 
html = response.text 
soup = BeautifulSoup(html, "html.parser")
team=[i.get_text() for i in soup.select('td')]
team=team[1::17][:10]
for i in range(len(team)) :
        if team[i]=='KIA' : team[i]='HT'
        if team[i]=='롯데' : team[i]='LT'
        if team[i]=='두산' : team[i]='OB'
        if team[i]=='키움' : team[i]='WO'
        if team[i]=='삼성' : team[i]='SS'
        if team[i]=='한화' : team[i]='HH'

# pitcher SF - score
url = "https://www.koreabaseball.com/Record/Team/Pitcher/Basic2.aspx"
response = requests.get(url) 
html = response.text 
soup = BeautifulSoup(html, "html.parser")
soup.get('td')
sf=[i.get_text() for i in soup.select('td')]
sf=sf[13::17][:10]

test_SF = pd.DataFrame({"SF":sf}, index=team)

In [ ]:
# merge whole SF & test set
home_test_x = pd.concat([home_test_x, test_SF], axis=1).astype(float)
away_test_x = pd.concat([away_test_x, test_SF], axis=1).astype(float)

# 홈 원정별 경기수
home_test_x['count_game']=home_test_x.WLS_HOLD+home_test_x.WLS_LOSE+home_test_x.WLS_WIN+home_test_x.WLS_SAVE
away_test_x['count_game']=away_test_x.WLS_HOLD+away_test_x.WLS_LOSE+away_test_x.WLS_WIN+away_test_x.WLS_SAVE

# 총 경기 수 
home_test_x['total_game']=home_test_x.count_game+away_test_x.count_game
away_test_x['total_game']=home_test_x.count_game+away_test_x.count_game

# 경기 횟수의 비율로 SF 계산
home_test_x["SF"]=(home_test_x.count_game/home_test_x.total_game)*home_test_x.SF
away_test_x["SF"]=(away_test_x.count_game/away_test_x.total_game)*away_test_x.SF

# 필요없는 열 drop
home_test_x = home_test_x.drop(['count_game','total_game','ERA','START_CK','CG_CK','WLS_WIN','WLS_LOSE','WLS_HOLD','WLS_SAVE'], axis=1)
away_test_x = away_test_x.drop(['count_game','total_game','ERA','START_CK','CG_CK','WLS_WIN','WLS_LOSE','WLS_HOLD','WLS_SAVE'], axis=1)

# INN2 변수 정수화
home_test_x.INN2 = (home_test_x.INN2//1)*3+(home_test_x.INN2%1)*10
away_test_x.INN2 = (away_test_x.INN2//1)*3+(away_test_x.INN2%1)*10

In [ ]:
home_test_x.to_csv('pitcher_home_test_x.csv')
away_test_x.to_csv('pitcher_away_test_x.csv')